# Import

In [2]:
import pickle

import numpy as np
from matplotlib.lines import Line2D
from peewee import fn

from factors.age_factor import AgeModel, AgeFactor
from global_vars import Category
from models import ComponentCpu, Component, ComponentGpu, ReferenceValue, Deal, DealComponent
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import stats

from factors.perf_factor import PerfFactor
from scrapers.component import BenchmarkSpider, UserBenchmarkSpider

# Parameters

In [3]:
ReferenceValue.GPU_FACTOR = 1.0
ReferenceValue.CPU_FACTOR = 1.0

model_file = 'models.pkl'
bn_name = BenchmarkSpider.name
ub_name = UserBenchmarkSpider.name

load_models = False
if load_models:
    with open(model_file, 'rb') as file:
        models = pickle.load(file)
    exit(0)

# Fixing missing values

In [4]:
# Fixing missing Userbenchmark
fix_missing = True
if fix_missing:
    for comp in [('GeForce GT 710', 3.33, 68), ('GeForce GT 1030', 14.8, 40),

                 ('GeForce GTX 560 Ti', -1, 116), ('GeForce GTX 760', -1, 87),
                 ('GeForce GTX 780', -1, 88), ('GeForce GTX 1050 Ti', -1, 47),
                 ('GeForce GTX 1060', -1, 50), ('GeForce GTX 1070', -1, 51),
                 ('GeForce GTX 1080', -1, 51),

                 ('GeForce RTX 2060', -1, 20), ('GeForce RTX 2070', -1, 23),
                 ('GeForce RTX 2080', -1, 24), ('GeForce GTX 1650', -1, 17)]:
        query = ComponentGpu.select().join(Component).where((Component.name == comp[0])).get()
        query.userbenchmark_score = max(query.userbenchmark_score, comp[1])
        query.age = max(query.age, comp[2])
        query.save()

# Price refs

In [5]:
# Name, price
cpu_ref = [('Intel Celeron G5900', 53.0),  # Low-end
           ('Intel Pentium Gold G5400', 70.0),  # Low
           ('Intel Core i3 9100F', 98.0),  # Mid-low
           ('Intel Core i3 10100', 160.0),  # Mid
           ('Intel Core i5 9600KF', 240.0),  # Mid-High
           ('Intel Core i7 9700KF', 380.0),  # High
           ('Intel Core i9 10900K', 610.0)   # High-end
           ]

# Name, price
gpu_ref = [('GeForce GT 710', 48.0),  # Low-end
           ('GeForce GT 1030', 90.0),  # Low
           ('GeForce GTX 1650', 170.0),  # Mid-low
           ('GeForce GTX 1660', 220.0),  # Mid
           ('GeForce RTX 2060', 340.0),  # Mid-High
           ('GeForce RTX 2070', 450.0),  # High
           ('GeForce RTX 3080', 850.0)   # High-end
           ]

# Price/perf // Perf models

In [ ]:
plot_box = True
models = []

for refs in [(Category.CPU, cpu_ref, ComponentCpu), (Category.GPU, gpu_ref, ComponentGpu)]:
    price_score_benchmark_net = []
    benchmark_net_scores = []
    price_score_userbenchmark = []
    userbenchmark_scores = []
    ages = []
    for ref in refs[1]:
        query = refs[2].select().join(Component).where((Component.name == ref[0])).get()
        if query.benchmark_net_score == 0 or query.userbenchmark_score == 0:
            print('{} do not have scores'.format(query.parent.name))
            exit(0)
        benchmark_net_scores.append(query.benchmark_net_score)
        userbenchmark_scores.append(query.userbenchmark_score)
        price_score_benchmark_net.append(ref[1] / benchmark_net_scores[-1])
        price_score_userbenchmark.append(ref[1] / userbenchmark_scores[-1])
        ages.append(query.age)

    for benchmark_site in [(benchmark_net_scores, price_score_benchmark_net, '{} on {}'.format(Category.labels[refs[0]], bn_name), bn_name),
                           (userbenchmark_scores, price_score_userbenchmark, '{} on {}'.format(Category.labels[refs[0]], ub_name), ub_name)]:
        x = benchmark_site[0]
        y = benchmark_site[1]

        xnew = np.linspace(min(x), max(x), num=100, endpoint=True)
        # f = InterpolatedUnivariateSpline(x, y)
        model = interp1d(x, y, kind='linear', fill_value='extrapolate')
        models.append(PerfFactor(benchmark_site[3], refs[0], model))

        legend_elements = [Line2D([0], [0], marker='o', color='b', label=str(x),
                                  markerfacecolor='b', markersize=5) for x in refs[1]]

        if plot_box:
            fig, ax = plt.subplots()
            ax.legend(handles=legend_elements)

            plt.plot(x, y, 'o', xnew, model(xnew), '-')
            plt.title(benchmark_site[2])
            plt.xlabel("Perf")
            plt.ylabel("Price / Perf")
            plt.show()

# Save models
with open(model_file, 'wb') as file:
    pickle.dump(models, file)

# TODO: best gpu/cpu pairs

In [7]:
# Cpu, gpu
# TODO: complete
best_pairs = [('', ''),  # Low-end
           ('', ''),  # Low
           ('', ''),  # Mid-low
           ('', ''),  # Mid
           ('', ''),  # Mid-High
           ('', ''),  # High
           ('', '')   # High-end
           ]

# TODO: gpu/cpu pairs graph / save model


# Second hands price refs

In [8]:
# ComponentGpu.select(Component.name, fn.AVG(Deal.full_price).alias('price_avg')).join(Component).join(DealComponent).join(Deal).where((fn.SUM(Deal) > 10) and (Deal.label == DealClass.REAL)).group_by(Component.name)

gpu_data = {'GeForce GTX 550 Ti': ([41.0, 40.0, 43.0, 75.0, 37.9, 47.0, 52.9, 37.9, 37.9, 33 + 12.9, 25+12.9,49+17.9,47.9+12.9,30+17,38+13,70+10,65+17,38+13,35+14,40+16,44+10,81+17]),
            'GeForce GTX 560 Ti': ([34+7.5,30+12,79+7,70+6,58+9,29+12.9,55+10,24.9+12.9,49.9+12.9,36+17,45+12.9,30+12,30+12.9,29+12.9,29+12.9,23.75+15,35+13,39+12.9,39+12.9,49+12.9,50+13]),
            'GeForce GTX 570': ([45+7,50+14,39+13,38+13,40+9,38+13,38+17,22+14,39+13,36+9,40+11,38+8.37,38+8.37,50+16,61+13,43+11,35+11.4]),
            'GeForce GTX 580': ([44+13,45+13,44+13,47+10,90+13,55+13,55+9,49+13,69+17,62+13,87+9,55+12,64.80+13,71.63+12,61.52+13,60+13,50+12,70+13,48+13]),

            'GeForce GTX 650': ([60+4.6,38+13,31+13,40+13,33+13,69+13,39,35+24,62+8,43+13,40+10,70+8,57+15,35+24,40+10]),
            'GeForce GTX 660': ([40+5,44+16,40+14,50+13,63+13,99+16,63+13,44+13,44+11,70+18,110+13,65+22,103+16,56+10,59+13,90+25,57+27,85+22]),
            'GeForce GTX 670': ([89+4,65+9,59+13,65+13,70+12,56+17,109+17,76+9]),
            'GeForce GTX 680': ([80+5,90+4,69+13,75+20,100+10,100+10]),

            'GeForce GTX 750 Ti': ([80+8,79+6,65+13,60+13,90+12,79+18,55+15,65+14,57+14,95+20,74+13,92+25,78+13,75+20]),
            'GeForce GTX 760': ([65+13,50+50,90+30,141+14,75+25,60+18]),
            'GeForce GTX 770': ([117+12,119+4,82,150+6,110,129,129+18,129+18,94+13,71+13,100+14,92+13,118+13,65+13,99+16,108,70+25,80+20,85+16,100+25,130+16,81+13,80+14,80+13]),
            'GeForce GTX 780': ([160+7.5,100+13,120+13,120+13,105,195+21,109+16,129+15.5,104+20,130+12,100+25,110+22,100+40,149+9,]),

            'GeForce GTX 960': ([70+15,100+110+5,109+7,120+13,110+14,128+13,109+13,86+13,90+13,71+10,87+19,95+13,109+21,80+7,148+25,75+15,70+18,119+13,99+13,90+15,86+13,82+7,92+12,89+14,130+20,95+12,92+12,85+38,101+13,130+47,90+12,86+13,80+12,88+13,78+12,90+12,169+29,141+13,95+13,99+13,100,155,206+15,70,100,113+10]),
            'GeForce GTX 970': ([109,110+5,220+6,120+5,120+4,120,155+5,140+15,153+8,119+13,114+13,94+26,196+16,152+10,208+18,119+13,98+14,163+16,109+20,160+25,128+17,115+15,195+16,120+40,120+25,140+16,180+20,120+25,124+13,129+21,108+10,157+15,250+7,250+9,157.6+16,229+20,168+20,188,197+10,120+10,140+9,139]),
            'GeForce GTX 980': ([175+5,140,150+10,225,290,152+17,145+13,200+20,178+13,175+21,299+8,152+17,219+13,163,178+39,140+10,166+13,165+19,165+15,170+30,235+20]),

            'GeForce GTX 1050 Ti': ([99+13,114+14,119+5,135+5,139,140+13,140,99+10,119+14,105+13,105+13,125+13,120+13,117+7,105+13,135+13,125+13,129+13,105+13,105+13,129+13,119+19,119+13,105+13,130+8,160,135+10,112+14,108+21,130+8,141+14,140+30,125+14,132+24,170+17,170+17,185+13,119+35,152+13,119+10,129+20,107+14,132+39,130+20,121+20,127+20,152+32,122+16,140+14,145+17,113+10,126+10,112+10,153+10]),
            'GeForce GTX 1060': ([149+12,160+25,190+13,150+5,172+18,150+7,160+15,210+10,149+15,255,137+14,155+13,159+13,195+20,145+16,164+13,216,195+8,221+7,165+30,200+10,160+15,163+11,176+5,200+17,162+21,140+21,145+21,229+18,152+13,189+15,195+17,189+20,195+15,240+13,181+9,176+30,190+25,195+13,175+36,230+20,188+18,228+12,195+23,200,200]),
            'GeForce GTX 1070': ([210,199+10,230+20,170+10,223,240,260,219+15,280+10,255+18,174+19,259+13,212+15,239+20,237+15,260+18,279+13,300+17,239+13,279+13,279+13,225,259+13,210+16,299+18,233+14,260+15,229+13,249+13,289+10,250+16,199+30,258+5,304+16,239+30,333+17,258+5,239+34,355+10,213+25,196+24,279+30,221+21,300+12,204+25,325+22,250+75,339+10,368,219+15,216+22,274+5+240+30,256+22,218+15,196+34,270+40,222+22,229+30,217+76,230+40,289+20,250+15,300+35,200+91,292+19,184+13,309,319+10,226+12,152+13,269+20,293+18,293+33,240+40,]),
            'GeForce GTX 1080': ([279+15,270+10,249+15,285+14,289,300+20,300+9,420+30,299+13,271+27,289+15,320+33,275+45,415+15,320+15,285+5,370,389+25,370,350+20,325+20,281+27,271+39,287+37,285+14,299+50,380+40,275+25,340+20,336+27,380+40,380+40,333+21,349+10,422+13,375+35,389,345+20,340+15,343+15,]),

            'GeForce GTX 1650': ([125+13,180+11,169+10,150+1,153+13]),
            'GeForce GTX 1660': ([179+9,180+20,230+6,215+17,227+11,180]),

            'GeForce RTX 2060': ([300,290+9,300,285+20,308+10,309+10]),
            'GeForce RTX 2070': ([320+15,370+10,350+10,400+15,399,444+15,411+38,438+13,480+18,438+18,387+10]),
            'GeForce RTX 2080': ([450+20,540+10,540,520+50,540+16,597+18,539+30,575+17,603+18,647+8,542+32,650+55,603+20,534+35,607+10]),

            }

# Plot data

In [ ]:
plot_box = True

x = dict()
x[bn_name] = []
x[ub_name] = []
y = dict()
y[bn_name] = dict()
y[bn_name]['q1'] = []
y[bn_name]['q2'] = []
y[bn_name]['q3'] = []
y[ub_name] = dict()
y[ub_name]['q1'] = []
y[ub_name]['q2'] = []
y[ub_name]['q3'] = []
ages = []

for data in gpu_data.items():
    query = ComponentGpu.select().join(Component).where((Component.name == data[0])).get()
    query.value_model = 'perf_model'
    ages.append(query.age)
    if query.age == -1:
        print('{} dont have age'.format(query.parent.name))
    if plot_box:
        fig, ax = plt.subplots()
        ax.set_title(data[0])
        ax.boxplot(data[1])
        plt.axhline(y=query.benchmark_net_value(), color='r', linestyle='-', label='Predicted value (net)')
        plt.axhline(y=query.userbenchmark_value(), color='b', linestyle='-', label='Predicted value (user)')
        ax.legend()
        fig.show()
    for quantile in [('q1',0.25), ('q2',0.5), ('q3',0.75)]:
        y[bn_name][quantile[0]].append(query.benchmark_net_value() / np.quantile(data[1], quantile[1]))
        y[ub_name][quantile[0]].append(query.userbenchmark_value() / np.quantile(data[1], quantile[1]))
    x[bn_name].append(query.benchmark_net_score)
    x[ub_name].append(query.userbenchmark_score)

    print('{}:{}, N:{}, U:{}'.format(data[0],
                                     ' ',
                                     # stats.describe(data[1]),
                                     query.benchmark_net_value() / np.median(data[1]),
                                     query.userbenchmark_value() / np.median(data[1])))

# Models for second hand factor

In [ ]:
def fit_func(x, a, b, c):
    return [a*x1**2+b*x1+c for x1 in x]
    # return a*x+b

for idx in [bn_name, ub_name]:
    for x_type in ['perf', 'age']:
        for quantile in y[idx].keys():

            if x_type == 'perf':
                plt.plot(x[idx], y[idx][quantile], 'o')
                plt.xlabel("Performance")
            else:
                # plt.plot(list(enumerate(range(len(y[idx][quantile])))), y[idx][quantile], 'o')
                plt.xlabel("Age (in months)")
                if quantile == 'q1':
                    xnew = np.linspace(min(ages), max(ages), num=100, endpoint=True)
                    model = AgeModel(ages, y[idx][quantile])
                    models.append(AgeFactor(idx, Category.GPU, model))
                    plt.plot(ages, y[idx][quantile], 'o', xnew, model(xnew), '-')
                else:
                    plt.plot(ages, y[idx][quantile], 'o')
            plt.ylabel("Benchmark value")
            plt.title('Benchmark {} value based on second hand data ({}, cor:{})'
                .format(idx, quantile, round(stats.spearmanr(x[idx], y[idx][quantile]).correlation, 3)))
            plt.show()


# Save models
with open(model_file, 'wb') as file:
    pickle.dump(models, file)
